# Final Submission <br>
Final submission for NLP Corsework 


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Pre Procesing

In [3]:
from pre_processing import PreProcessing
pre_process = PreProcessing()

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\giova\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
reviews = pd.read_csv('reviews.csv')

In [5]:
## Zipf's Law Implementation
processed = pre_process.set_n_grams(3, pre_process.zipfs_law(0.2, pre_process.lemmatize(pre_process.lowercase(pre_process.remove_stopwords(pre_process.remove_punctuation(reviews.review))))))

## No Zipf's Law Implementation only do fo n_grams = 1
# processed = pre_process.set_n_grams(1, pre_process.stem(pre_process.remove_stopwords(pre_process.lowercase(pre_process.remove_punctuation(reviews.review)))))

## TFIDF

In [ ]:
from tfidf_class_to_analyze import TFIDF
tfidf = TFIDF()

In [ ]:
review_terms = {}
#label_terms = []
for review_id in range(len(processed)):
    review_terms[review_id] = tfidf.get_terms(processed[review_id])

all_terms = tfidf.collect_vocabulary(review_terms)

#doc_vectors = tfidf.vectorize(review_terms, all_terms)

doc_idfs = tfidf.calculate_idfs(all_terms, review_terms)
doc_vectors = tfidf.vectorize_idf(review_terms, doc_idfs, all_terms)

## Train, Test, Dev Splits

In [ ]:
X_train, X_test, X_dev, y_train, y_test, y_dev = pre_process.set_data_splits(doc_vectors, list(reviews.label))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

## Naive Bayes - own implementation

In [ ]:
from nb_classifier_2 import NB_Classifier
nb = NB_Classifier(alpha=1)

In [ ]:
nb.fit(np.array(X_train), y_train)

MemoryError: Unable to allocate 2.89 GiB for an array with shape (1422, 272343) and data type float64

In [ ]:
pred = nb.predict(np.array(X_train))

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_train, pred))
print("Accuracy:", accuracy_score(y_train, pred))
print("Precision:", precision_score(y_train, pred))
print("Recall:", recall_score(y_train, pred))
print("F1 Score:", f1_score(y_train, pred))


Confusion Matrix:
 [[1422    0]
 [   0 1378]]
Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
pred_test = nb.predict(np.array(X_test))

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_test))
print("Accuracy:", accuracy_score(y_test, pred_test))
print("Precision:", precision_score(y_test, pred_test))
print("Recall:", recall_score(y_test, pred_test))
print("F1 Score:", f1_score(y_test, pred_test))


Confusion Matrix:
 [[239  62]
 [ 81 218]]
Accuracy: 0.7616666666666667
Precision: 0.7785714285714286
Recall: 0.7290969899665551
F1 Score: 0.7530224525043179


## Naive Bayes - sklearn

In [41]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

mnb.fit(X_train, y_train)
pred = mnb.predict(X_train)

In [42]:
# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_train, pred))
print("Accuracy:", accuracy_score(y_train, pred))
print("Precision:", precision_score(y_train, pred))
print("Recall:", recall_score(y_train, pred))
print("F1 Score:", f1_score(y_train, pred))

Confusion Matrix:
 [[1203  219]
 [ 202 1176]]
Accuracy: 0.8496428571428571
Precision: 0.843010752688172
Recall: 0.8534107402031931
F1 Score: 0.8481788676523622


In [43]:
pred = mnb.predict(X_test)

# Assuming 'y_true' is the true labels and 'y_pred' is the predicted labels
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred))

Confusion Matrix:
 [[245  56]
 [ 55 244]]
Accuracy: 0.815
Precision: 0.8133333333333334
Recall: 0.8160535117056856
F1 Score: 0.8146911519198664


## SGD

### Logistic Regression

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_log = SGDClassifier(loss='log', max_iter=1000, tol=1e-3, random_state=42)
sgd_log.fit(X_train, y_train)

In [ ]:
predicts = sgd_log.predict(X_train)

print("Confusion Matrix:\n", confusion_matrix(y_train, predicts))
print("Accuracy:", accuracy_score(y_train, predicts))
print("Precision:", precision_score(y_train, predicts))
print("Recall:", recall_score(y_train, predicts))
print("F1 Score:", f1_score(y_train, predicts))

In [53]:
predicts = sgd_log.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, predicts))
print("Accuracy:", accuracy_score(y_test, predicts))
print("Precision:", precision_score(y_test, predicts))
print("Recall:", recall_score(y_test, predicts))
print("F1 Score:", f1_score(y_test, predicts))

Confusion Matrix:
 [[233  68]
 [ 62 237]]
Accuracy: 0.7833333333333333
Precision: 0.7770491803278688
Recall: 0.7926421404682275
F1 Score: 0.7847682119205298


### SVM

In [54]:
sgd_svm = SGDClassifier(loss='hinge', max_iter=1000, tol=1e-3, random_state=42)
sgd_svm.fit(X_train, y_train)

SVC(kernel='linear')

In [55]:
outp = sgd_svm.predict(X_train)

print("Confusion Matrix:\n", confusion_matrix(y_train, outp))
print("Accuracy:", accuracy_score(y_train, outp))
print("Precision:", precision_score(y_train, outp))
print("Recall:", recall_score(y_train, outp))
print("F1 Score:", f1_score(y_train, outp))

Confusion Matrix:
 [[1419    3]
 [   2 1376]]
Accuracy: 0.9982142857142857
Precision: 0.9978245105148659
Recall: 0.9985486211901307
F1 Score: 0.9981864345302865


In [56]:
predicts = sgd_svm.predict(X_test)

print("Confusion Matrix:\n", confusion_matrix(y_test, predicts))
print("Accuracy:", accuracy_score(y_test, predicts))
print("Precision:", precision_score(y_test, predicts))
print("Recall:", recall_score(y_test, predicts))
print("F1 Score:", f1_score(y_test, predicts))

Confusion Matrix:
 [[226  75]
 [ 72 227]]
Accuracy: 0.755
Precision: 0.7516556291390728
Recall: 0.7591973244147158
F1 Score: 0.7554076539101496


## Bert

Do not run any of these cells it's quite messy. It saves the bert model locally so we can analyze performance in next section

### Preprocessing

In [5]:
## Imports for BERT model
from transformers import BertTokenizerFast
from transformers import DataCollatorWithPadding
import evaluate
from transformers import AutoModelForSequenceClassification

from transformers import BertForSequenceClassification, Trainer, TrainingArguments

from transformers import pipeline

from transformers import AutoTokenizer

In [6]:
## Test. train dev split using same method as above
X_train, X_test, X_dev, y_train, y_test, y_dev = pre_process.set_data_splits(reviews.review, reviews.label)

In [7]:
## Which version are you using?
#assert 1 == 2

#tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
accuracy = evaluate.load("accuracy")

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [11]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

<b> Data processing

In [12]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
val_encodings = tokenizer(list(X_dev), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

In [13]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, list(y_train))
val_dataset = IMDbDataset(val_encodings, list(y_dev))
test_dataset = IMDbDataset(test_encodings, list(y_test))

In [14]:
# from huggingface_hub import notebook_login
# notebook_login()

### BERT Uncased

In [7]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

NameError: name 'id2label' is not defined

In [24]:
## NO Metrics Compute

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    no_cuda= False
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased").to('cuda')

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset

)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/525 [00:00<?, ?it/s]

{'loss': 0.7087, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.06}
{'loss': 0.7079, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.11}
{'loss': 0.7217, 'learning_rate': 3e-06, 'epoch': 0.17}
{'loss': 0.6852, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.23}
{'loss': 0.6813, 'learning_rate': 5e-06, 'epoch': 0.29}
{'loss': 0.6638, 'learning_rate': 6e-06, 'epoch': 0.34}
{'loss': 0.6478, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.4}
{'loss': 0.6207, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.46}
{'loss': 0.5881, 'learning_rate': 9e-06, 'epoch': 0.51}
{'loss': 0.5614, 'learning_rate': 1e-05, 'epoch': 0.57}
{'loss': 0.556, 'learning_rate': 1.1000000000000001e-05, 'epoch': 0.63}
{'loss': 0.3925, 'learning_rate': 1.2e-05, 'epoch': 0.69}
{'loss': 0.3736, 'learning_rate': 1.3000000000000001e-05, 'epoch': 0.74}
{'loss': 0.4339, 'learning_rate': 1.4000000000000001e-05, 'epoch': 0.8}
{'loss': 0.3734, 'learning_rate': 1.5e-05, 'epoch': 0.86}
{'loss': 0.3617, 'learning

TrainOutput(global_step=525, training_loss=0.3391809720084781, metrics={'train_runtime': 6857.364, 'train_samples_per_second': 1.225, 'train_steps_per_second': 0.077, 'train_loss': 0.3391809720084781, 'epoch': 3.0})

In [19]:
## Metrics computed

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

#trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
trainer.train()

  0%|          | 0/350 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.28091785311698914, 'eval_accuracy': 0.9, 'eval_runtime': 25.9731, 'eval_samples_per_second': 23.101, 'eval_steps_per_second': 1.463, 'epoch': 1.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.3185114562511444, 'eval_accuracy': 0.9, 'eval_runtime': 26.1432, 'eval_samples_per_second': 22.951, 'eval_steps_per_second': 1.454, 'epoch': 2.0}
{'train_runtime': 4686.3677, 'train_samples_per_second': 1.195, 'train_steps_per_second': 0.075, 'train_loss': 0.3038702828543527, 'epoch': 2.0}


TrainOutput(global_step=350, training_loss=0.3038702828543527, metrics={'train_runtime': 4686.3677, 'train_samples_per_second': 1.195, 'train_steps_per_second': 0.075, 'train_loss': 0.3038702828543527, 'epoch': 2.0})

#### Load and evaluation

In [43]:
#tokenizer = AutoTokenizer.from_pretrained("results/checkpoint-500/")
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained("results/checkpoint-500/")

In [48]:
pred = []
## Change here for train/test/dev set
for w in list(X_test):
    inputs = tokenizer(w, return_tensors="pt", truncation= True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    pred.append(1 if model.config.id2label[predicted_class_id] == 'LABEL_1' else 0)
    #print(model.config.id2label[predicted_class_id])

In [46]:
print("Confusion Matrix:\n", confusion_matrix(y_train, pred))
print("Accuracy:", accuracy_score(y_train, pred))
print("Precision:", precision_score(y_train, pred))
print("Recall:", recall_score(y_train, pred))
print("F1 Score:", f1_score(y_train, pred))

ValueError: Found input variables with inconsistent numbers of samples: [2800, 600]

In [50]:
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred))

Confusion Matrix:
 [[287  14]
 [ 63 236]]
Accuracy: 0.8716666666666667
Precision: 0.944
Recall: 0.7892976588628763
F1 Score: 0.8597449908925319


### Cased Bert

In [15]:
cased_model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:

training_args = TrainingArguments(
    output_dir="./results/cased",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

cased_model = BertForSequenceClassification.from_pretrained("bert-base-cased") #.to('cuda')

trainer = Trainer(
    model=cased_model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/350 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.2918759286403656, 'eval_accuracy': 0.8833333333333333, 'eval_runtime': 26.5647, 'eval_samples_per_second': 22.586, 'eval_steps_per_second': 1.43, 'epoch': 1.0}


  0%|          | 0/38 [00:00<?, ?it/s]

{'eval_loss': 0.30336907505989075, 'eval_accuracy': 0.8883333333333333, 'eval_runtime': 26.4312, 'eval_samples_per_second': 22.7, 'eval_steps_per_second': 1.438, 'epoch': 2.0}
{'train_runtime': 4622.5317, 'train_samples_per_second': 1.211, 'train_steps_per_second': 0.076, 'train_loss': 0.3330478777204241, 'epoch': 2.0}


TrainOutput(global_step=350, training_loss=0.3330478777204241, metrics={'train_runtime': 4622.5317, 'train_samples_per_second': 1.211, 'train_steps_per_second': 0.076, 'train_loss': 0.3330478777204241, 'epoch': 2.0})

#### Loading and Evaluation

In [52]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
imported_model = AutoModelForSequenceClassification.from_pretrained('results/cased/checkpoint-350/')

In [53]:
predicts = []
# Change here for train/test/dev split
for w in list(X_test):
    inputs = tokenizer(w, return_tensors="pt", truncation= True)
    with torch.no_grad():
        logits = imported_model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    predicts.append(1 if imported_model.config.id2label[predicted_class_id]== 'LABEL_1' else 0)
    #print(model.config.id2label[predicted_class_id])

In [54]:
print("Confusion Matrix:\n", confusion_matrix(y_test, pred))
print("Accuracy:", accuracy_score(y_test, pred))
print("Precision:", precision_score(y_test, pred))
print("Recall:", recall_score(y_test, pred))
print("F1 Score:", f1_score(y_test, pred))

Confusion Matrix:
 [[287  14]
 [ 63 236]]
Accuracy: 0.8716666666666667
Precision: 0.944
Recall: 0.7892976588628763
F1 Score: 0.8597449908925319
